# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 18:07:42.988000 4185785 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:07:42.988000 4185785 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:07:44] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35176, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=27885

[2025-08-11 18:07:44] Using default HuggingFace chat template with detected content format: string


W0811 18:07:52.534000 4186591 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:07:52.534000 4186591 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 18:07:52.542000 4186592 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:07:52.542000 4186592 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:07:54] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-11 18:07:54] Init torch distributed begin.


...

Capturing batches (bs=3 avail_mem=53.68 GB):  25%|██▌       | 1/4 [00:04<00:14,  4.81s/it]

Capturing batches (bs=1 avail_mem=53.66 GB):  50%|█████     | 2/4 [00:05<00:04,  2.46s/it]

Capturing batches (bs=1 avail_mem=53.66 GB): 100%|██████████| 4/4 [00:09<00:00,  2.34s/it]
[2025-08-11 18:08:27] Capture draft cuda graph end. Time elapsed: 9.87 s. mem usage=-3.78 GB. avail mem=53.64 GB.
[2025-08-11 18:08:27] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.64 GB


Capturing batches (bs=1 avail_mem=53.56 GB): 100%|██████████| 4/4 [00:00<00:00, 111.14it/s]
[2025-08-11 18:08:28] Capture draft extend cuda graph end. Time elapsed: 0.64 s. mem usage=0.08 GB. avail mem=53.56 GB.
[2025-08-11 18:08:28] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.56 GB
[2025-08-11 18:08:28] INFO:     Started server process [4185785]
[2025-08-11 18:08:28] INFO:     Waiting for application startup.
[2025-08-11 18:08:28] INFO:     Application startup complete.
[2025-08-11 18:08:28] INFO:     Uvicorn running on http://127.0.0.1:35176 (Press CTRL+C to quit)


[2025-08-11 18:08:29] INFO:     127.0.0.1:42266 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-11 18:08:29] INFO:     127.0.0.1:42276 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 18:08:29] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 18:09:19] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-11 18:09:19] INFO:     127.0.0.1:42286 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 18:09:19] The server is fired up and ready to roll!


[2025-08-11 18:09:20] INFO:     127.0.0.1:43516 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 18:09:27.300000 4193461 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:09:27.300000 4193461 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:09:29] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=35271, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=6465106

[2025-08-11 18:09:29] Using default HuggingFace chat template with detected content format: string


W0811 18:09:36.672000 4193736 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:09:36.672000 4193736 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 18:09:36.715000 4193737 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:09:36.715000 4193737 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:09:37] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-11 18:09:37] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.91 GB): 100%|██████████| 4/4 [00:15<00:00,  3.85s/it]
[2025-08-11 18:10:41] Capture draft cuda graph end. Time elapsed: 16.14 s. mem usage=10.33 GB. avail mem=28.40 GB.
[2025-08-11 18:10:41] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=28.40 GB


Capturing batches (bs=1 avail_mem=5.68 GB): 100%|██████████| 4/4 [00:00<00:00, 31.38it/s]
[2025-08-11 18:10:42] Capture draft extend cuda graph end. Time elapsed: 1.54 s. mem usage=22.72 GB. avail mem=5.67 GB.
[2025-08-11 18:10:42] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=5.67 GB


[2025-08-11 18:10:43] INFO:     Started server process [4193461]
[2025-08-11 18:10:43] INFO:     Waiting for application startup.
[2025-08-11 18:10:43] INFO:     Application startup complete.
[2025-08-11 18:10:43] INFO:     Uvicorn running on http://127.0.0.1:35271 (Press CTRL+C to quit)


[2025-08-11 18:10:43] INFO:     127.0.0.1:47082 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-11 18:10:44] INFO:     127.0.0.1:47090 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 18:10:44] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 18:10:46] INFO:     127.0.0.1:47100 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 18:10:46] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 18:10:48] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 18:10:49] INFO:     127.0.0.1:47108 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 18:10:56.525000 8713 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:10:56.525000 8713 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:10:58] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39219, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-11 18:10:58] Using default HuggingFace chat template with detected content format: string


W0811 18:11:05.698000 9040 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:11:05.698000 9040 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 18:11:05.779000 9041 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:11:05.779000 9041 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:11:06] Casting torch.bfloat16 to torch.float16.


...

[2025-08-11 18:11:37] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=38.98 GB


[2025-08-11 18:11:38] INFO:     Started server process [8713]
[2025-08-11 18:11:38] INFO:     Waiting for application startup.
[2025-08-11 18:11:38] INFO:     Application startup complete.
[2025-08-11 18:11:38] INFO:     Uvicorn running on http://127.0.0.1:39219 (Press CTRL+C to quit)


[2025-08-11 18:11:38] INFO:     127.0.0.1:59406 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-11 18:11:39] INFO:     127.0.0.1:59414 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 18:11:39] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 18:11:39] INFO:     127.0.0.1:59420 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 18:11:39] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 18:11:43] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 18:11:44] INFO:     127.0.0.1:57654 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 18:11:51.512000 11889 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:11:51.512000 11889 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:11:53] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34552, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-11 18:11:53] Using default HuggingFace chat template with detected content format: string


W0811 18:12:01.549000 12261 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:12:01.549000 12261 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 18:12:01.653000 12260 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:12:01.653000 12260 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:12:02] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=58.24 GB): 100%|██████████| 4/4 [00:05<00:00,  1.34s/it]
[2025-08-11 18:12:30] Capture draft cuda graph end. Time elapsed: 5.91 s. mem usage=-0.14 GB. avail mem=58.20 GB.
[2025-08-11 18:12:30] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.20 GB


Capturing batches (bs=1 avail_mem=58.08 GB): 100%|██████████| 4/4 [00:00<00:00, 23.42it/s]
[2025-08-11 18:12:32] Capture draft extend cuda graph end. Time elapsed: 1.77 s. mem usage=0.11 GB. avail mem=58.08 GB.
[2025-08-11 18:12:32] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.08 GB


[2025-08-11 18:12:33] INFO:     Started server process [11889]
[2025-08-11 18:12:33] INFO:     Waiting for application startup.
[2025-08-11 18:12:33] INFO:     Application startup complete.
[2025-08-11 18:12:33] INFO:     Uvicorn running on http://127.0.0.1:34552 (Press CTRL+C to quit)


[2025-08-11 18:12:33] INFO:     127.0.0.1:58642 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-11 18:12:34] INFO:     127.0.0.1:58652 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 18:12:34] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 18:12:34] INFO:     127.0.0.1:58654 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 18:12:34] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-11 18:12:38] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 18:12:39] INFO:     127.0.0.1:58656 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0811 18:12:46.036000 16291 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:12:46.036000 16291 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:12:48] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=38945, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=514907823, constrained_jso

[2025-08-11 18:12:48] Using default HuggingFace chat template with detected content format: string


W0811 18:12:55.316000 17249 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:12:55.316000 17249 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0811 18:12:55.316000 17248 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0811 18:12:55.316000 17248 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-11 18:12:56] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-11 18:12:56] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=58.94 GB): 100%|██████████| 4/4 [00:02<00:00,  1.76it/s]
[2025-08-11 18:13:09] Capture draft cuda graph end. Time elapsed: 2.77 s. mem usage=0.51 GB. avail mem=58.94 GB.
[2025-08-11 18:13:09] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=58.94 GB


Capturing batches (bs=1 avail_mem=52.54 GB): 100%|██████████| 4/4 [00:00<00:00, 57.60it/s]
[2025-08-11 18:13:09] Capture draft extend cuda graph end. Time elapsed: 0.68 s. mem usage=10.24 GB. avail mem=48.70 GB.
[2025-08-11 18:13:09] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=48.37 GB


[2025-08-11 18:13:10] INFO:     Started server process [16291]
[2025-08-11 18:13:10] INFO:     Waiting for application startup.
[2025-08-11 18:13:10] INFO:     Application startup complete.
[2025-08-11 18:13:10] INFO:     Uvicorn running on http://0.0.0.0:38945 (Press CTRL+C to quit)


[2025-08-11 18:13:11] INFO:     127.0.0.1:32938 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-11 18:13:11] INFO:     127.0.0.1:32946 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-11 18:13:11] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-11 18:13:11] INFO:     127.0.0.1:32960 - "POST /generate HTTP/1.1" 200 OK
[2025-08-11 18:13:11] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-11 18:13:16] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-11 18:13:16] Decode batch. #running-req: 1, #token: 85, token usage: 0.00, accept len: 1.62, cuda graph: True, gen throughput (token/s): 9.81, #queue-req: 0, 


[2025-08-11 18:13:17] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, accept len: 1.70, cuda graph: True, gen throughput (token/s): 153.63, #queue-req: 0, 


[2025-08-11 18:13:17] Decode batch. #running-req: 1, #token: 219, token usage: 0.01, accept len: 1.65, cuda graph: True, gen throughput (token/s): 150.94, #queue-req: 0, 


[2025-08-11 18:13:17] Decode batch. #running-req: 1, #token: 286, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 149.69, #queue-req: 0, 


[2025-08-11 18:13:18] Decode batch. #running-req: 1, #token: 349, token usage: 0.02, accept len: 1.57, cuda graph: True, gen throughput (token/s): 141.08, #queue-req: 0, 


[2025-08-11 18:13:18] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, accept len: 1.65, cuda graph: True, gen throughput (token/s): 150.17, #queue-req: 0, 


[2025-08-11 18:13:19] Decode batch. #running-req: 1, #token: 480, token usage: 0.02, accept len: 1.62, cuda graph: True, gen throughput (token/s): 143.96, #queue-req: 0, 


[2025-08-11 18:13:19] Decode batch. #running-req: 1, #token: 550, token usage: 0.03, accept len: 1.75, cuda graph: True, gen throughput (token/s): 160.77, #queue-req: 0, 
[2025-08-11 18:13:19] INFO:     127.0.0.1:44430 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).